## Step 1 from Assignment
1. Load the global weather data into your big data technology of choice.
2. Join the stationlist.csv with the countrylist.csv to get the full country name
for each station number.
3. Join the global weather data with the full country names by station number.

In [1]:
#import pyspark spark session
from pyspark.sql import SparkSession

#create spark session
#set app name to the assigment challenge
spark = SparkSession.\
        builder.\
        appName("de-weather-challenge").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

In [ ]:
# step 1.1 load data
station_df = spark.read.csv(path="../../data/stationlist.csv", sep=",", header=True)

In [ ]:
station_df.